In [1]:
import numpy as np
import os

from PIL import Image

PATH = "../benchmarks_2/id_assumption_scaling"
ASSUMPTION_SIGNATURE = "a"
CUSTOM_UNSAT_HEAD = ""

In [20]:
n_assumptions = 10000
n_mucs = 10

density = (0.5 / (n_assumptions / 100))
verbose = 0

mucs = []

while True:
    assignment = np.random.binomial(n=1, p=density, size=n_assumptions)
    candidate = {i + 1 for i, x in enumerate(assignment) if x == 1}
    if sum(assignment) != 50:
        continue

    if verbose:
        print(assignment, candidate)

    if any([candidate.issubset(muc) or muc.issubset(candidate) for muc in mucs]):
        if verbose:
            print("DENIED")
    else:
        if verbose:
            print("APPROVED")
        mucs.append(candidate)
        if len(mucs) >= n_mucs:
            break

In [21]:
print(len(mucs))
for muc in mucs:
    print(muc)

assumption_dir = f"{n_assumptions}_assumptions"
if PATH:
    assumption_dir = f"{PATH}/{assumption_dir}"
muc_dir = f"{assumption_dir}/{n_mucs}_mucs"
dirname = muc_dir

if not os.path.isdir(assumption_dir):
    os.mkdir(assumption_dir)
if not os.path.isdir(muc_dir):
    os.mkdir(muc_dir)


img_width = 1280
img_height = int((n_mucs / n_assumptions) * img_width)
img = np.zeros(shape=(n_mucs, n_assumptions), dtype=np.uint8) + 255
for m, muc in enumerate(mucs):
    for a in muc:
        img[m, a - 1] = 0
Image.fromarray(img).resize((img_width, img_height), resample=Image.BOX).save(f"{dirname}/visualization.jpeg")

with open(f"{dirname}/encoding.lp", "w") as f:
    f.write("% ---[ AUTO GENERATED MUC BENCHMARK ]\n")
    f.write(f"% ---[ assumptions : {n_assumptions} ]\n")
    f.write(f"% ---[ mucs : {n_mucs} ]\n\n")

    f.write(f"num(1..{n_assumptions}).\n")
    f.write("{" + ASSUMPTION_SIGNATURE + "(X): num(X)}.\n\n")

    for muc in mucs:
        f.write(f"{CUSTOM_UNSAT_HEAD}:- " + ", ".join([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) + ".\n")

with open(f"{dirname}/assumptions.lp", "w") as f:
    for a in range(n_assumptions):
        f.write(f"assume({ASSUMPTION_SIGNATURE}({a + 1})).\n")

with open(f"{dirname}/results.txt", "w") as f:
    results = dict()
    results["minimal"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in mucs]

    minimum = [muc for muc in mucs if len(muc) == len(min(mucs, key=lambda x: len(x)))]
    results["minimum"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in minimum]

    f.write(str(results).replace("'", '"'))

with open(f"{assumption_dir}/README.md", "w") as f:
    f.write(f"# Random MUC Benchmarks [{n_assumptions} Assumptions]\n\n")
    for s, subdir in enumerate(os.walk(assumption_dir)):
        if subdir[0] != assumption_dir:
            subdir_mucs = int(subdir[0].split("/")[-1].replace("_mucs", ""))
            f.write(f"## {s} : {subdir_mucs} MUCs\n\n")
            f.write(f"![]({subdir[0].split('/')[-1]}/visualization.jpeg)\n\n")

10
{904, 2571, 3980, 6284, 8207, 5392, 4497, 7312, 8338, 8465, 152, 7837, 6432, 1698, 9763, 6309, 7719, 7208, 4649, 1962, 9132, 7728, 5810, 7225, 5051, 1084, 2750, 7742, 3905, 322, 6210, 1223, 2375, 5705, 6734, 1615, 9426, 3674, 6490, 1117, 4832, 3042, 7651, 8168, 745, 7787, 750, 4976, 7024, 6780}
{3587, 7556, 9349, 7814, 5128, 4620, 15, 9361, 274, 1557, 7958, 922, 5787, 1948, 7579, 798, 8350, 8481, 8870, 5801, 7210, 5292, 6704, 3638, 4154, 6079, 9408, 705, 4293, 6988, 9678, 7762, 2134, 9177, 351, 9567, 2273, 7265, 748, 3437, 6509, 6639, 2288, 3696, 8050, 9970, 1013, 7417, 4346, 9599}
{7424, 3202, 4098, 3717, 8584, 3721, 5004, 7181, 3598, 5390, 9232, 6676, 3477, 7710, 3491, 2725, 1062, 5547, 684, 1836, 3373, 7854, 7097, 2749, 2877, 7997, 5952, 9021, 6342, 7241, 8523, 1234, 8148, 8666, 733, 2784, 8032, 4324, 2533, 2663, 1384, 5737, 8552, 4715, 8810, 4206, 626, 1650, 8434, 4856}
{1543, 4487, 1676, 6545, 8724, 407, 6299, 7451, 9115, 545, 1827, 7723, 6195, 7102, 3775, 1984, 5568, 836, 9286